In [1]:
import pandas as pd
movies=pd.read_csv('movies.csv')

### 1.数据清洗

In [3]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140502 entries, 0 to 140501
Data columns (total 21 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   MOVIE_ID       140502 non-null  int64  
 1   NAME           140502 non-null  object 
 2   ALIAS          30322 non-null   object 
 3   ACTORS         82851 non-null   object 
 4   COVER          50654 non-null   object 
 5   DIRECTORS      70244 non-null   object 
 6   DOUBAN_SCORE   140502 non-null  float64
 7   DOUBAN_VOTES   140502 non-null  float64
 8   GENRES         136452 non-null  object 
 9   IMDB_ID        113256 non-null  object 
 10  LANGUAGES      131442 non-null  object 
 11  MINS           140502 non-null  float64
 12  OFFICIAL_SITE  9821 non-null    object 
 13  REGIONS        136501 non-null  object 
 14  RELEASE_DATE   77687 non-null   object 
 15  SLUG           140502 non-null  object 
 16  STORYLINE      87052 non-null   object 
 17  TAGS           97398 non-null

删除无用列并只保留评分大于6.5的电影

In [4]:
movies["GENRES"].head(10)

0       剧情/爱情
1       动作/爱情
2          剧情
3          爱情
4       剧情/历史
5       剧情/爱情
6          悬疑
7          科幻
8    动作/悬疑/古装
9          科幻
Name: GENRES, dtype: object

In [5]:
movies = movies.drop(
    columns=['COVER','IMDB_ID','MINS','OFFICIAL_SITE','RELEASE_DATE','SLUG','ACTOR_IDS','DIRECTOR_IDS','LANGUAGES','GENRES','ALIAS']
)
movies = movies[movies['DOUBAN_SCORE'] >= 6.5]

In [6]:
movies.isnull().sum()

MOVIE_ID           0
NAME               0
ACTORS          2023
DIRECTORS       2688
DOUBAN_SCORE       0
DOUBAN_VOTES       0
REGIONS            0
STORYLINE        492
TAGS               0
YEAR               0
dtype: int64

筛选评分人数大于5000的电影并降序排列

In [7]:
movies_new = movies[(movies['DOUBAN_VOTES'] >= 3000)].sort_values(by=['DOUBAN_SCORE','DOUBAN_VOTES'], ascending=[False,False])[['ACTORS','DIRECTORS','MOVIE_ID','NAME','DOUBAN_SCORE','STORYLINE','TAGS','REGIONS','YEAR']]

### 2.余弦相似度模型构建

In [8]:
movies_new['INFO'] = (
    movies['STORYLINE'].fillna('').astype(str) + " " +
    movies['TAGS'].fillna('').astype(str) + " " +
    movies['REGIONS'].fillna('').astype(str)
#    + " " +
#    movies['DIRECTORS'].fillna('').astype(str) + " " +
#    movies['ACTORS'].fillna('').astype(str) + " " +
#    movies['YEAR'].fillna('').astype(str)
)
movies_new = movies_new.drop(columns=['STORYLINE', 'TAGS', 'REGIONS'
                                      #, 'DIRECTORS', 'ACTORS', 'YEAR'
                                     ])

In [9]:
movies_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3886 entries, 64156 to 40584
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ACTORS        3825 non-null   object 
 1   DIRECTORS     3744 non-null   object 
 2   MOVIE_ID      3886 non-null   int64  
 3   NAME          3886 non-null   object 
 4   DOUBAN_SCORE  3886 non-null   float64
 5   YEAR          3886 non-null   float64
 6   INFO          3886 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 242.9+ KB


In [10]:
#添加中文分词库
import jieba
from sklearn.feature_extraction.text import CountVectorizer

C:\Users\wangyucheng\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\jieba\_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [11]:
# 中文分词器
def chinese_tokenizer(text):
    return jieba.lcut(str(text))
stopwords = [
    "的", "了", "在", "是", "我", "有", "和", "就", "不", "人", 
    "都", "一", "一个", "上", "也", "很", "到", "说", "要", "去",
    "你", "会", "着", "没有", "看", "好", "自己", "这", "那", 
    "为", "之", "对", "与", "而", "并", "等", "被", "及", "或",
    "但", "所以", "如果", "因为", "然后", "而且", "那么", "他们", 
    "我们", "你们", "它们", "什么", "哪个", "哪些", "哪里", "时候"
]
cv = CountVectorizer(
    max_features=5000,
    tokenizer=chinese_tokenizer,
    stop_words=stopwords,   # 中文停用词表（如果有就传 list）
    token_pattern=None      # 必须设 None，否则 tokenizer 会被覆盖
)

vector = cv.fit_transform(movies_new['INFO'].astype(str)).toarray()

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\WANGYU~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.364 seconds.
Prefix dict has been built successfully.


In [12]:
vector.shape

(3886, 5000)

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
similarity=cosine_similarity(vector)

In [15]:
similarity

array([[1.        , 0.34076584, 0.39717755, ..., 0.4830387 , 0.2358598 ,
        0.25715143],
       [0.34076584, 1.        , 0.71652678, ..., 0.64992399, 0.70318363,
        0.70137366],
       [0.39717755, 0.71652678, 1.        , ..., 0.62121336, 0.63570436,
        0.63871908],
       ...,
       [0.4830387 , 0.64992399, 0.62121336, ..., 1.        , 0.58649709,
        0.60962353],
       [0.2358598 , 0.70318363, 0.63570436, ..., 0.58649709, 1.        ,
        0.62765135],
       [0.25715143, 0.70137366, 0.63871908, ..., 0.60962353, 0.62765135,
        1.        ]], shape=(3886, 3886))

In [16]:
movies_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3886 entries, 64156 to 40584
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ACTORS        3825 non-null   object 
 1   DIRECTORS     3744 non-null   object 
 2   MOVIE_ID      3886 non-null   int64  
 3   NAME          3886 non-null   object 
 4   DOUBAN_SCORE  3886 non-null   float64
 5   YEAR          3886 non-null   float64
 6   INFO          3886 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 242.9+ KB


In [18]:
import numpy as np

def recommand(movie_name, topk=5):
    label_idx = movies_new.index[movies_new['NAME'] == movie_name]
    if len(label_idx) == 0:
        print("未找到该影片")
        return
    pos = movies_new.index.get_loc(label_idx[0])

    sims = similarity[pos]
    cand = np.argsort(-sims)
    cand = cand[cand != pos][:topk]

    recs = []
    for j in cand:
        recs.append({
            "电影名": movies_new.iloc[j]['NAME'],
            "豆瓣评分": movies_new.iloc[j]['DOUBAN_SCORE'],
        })
    df = pd.DataFrame(recs)
    return df

In [32]:
recommand("八部半")

,电影名,豆瓣评分
0,控制的极限,6.8
1,全面回忆,7.0
2,质数的孤独,7.0
3,窈窕淑男,8.0
4,尤里西斯的凝视,9.0


## 根据电影集进行推荐，输出为文字，简单排本

In [20]:
def recommend_from_collection(movie_names, topk=10, min_similarity=0.0, strategy='average'):
    """
    根据电影集合推荐相似电影
    
    Parameters:
    -----------
    movie_names : list
        电影名称列表
    topk : int
        推荐数量
    min_similarity : float
        最小相似度阈值
    strategy : str
        推荐策略: 'average'(平均相似度), 'maximum'(最大相似度), 'union'(并集推荐)
    
    Returns:
    --------
    pandas.DataFrame
        推荐结果
    """
    
    # 1. 验证输入电影
    valid_movies = []
    invalid_movies = []
    
    for movie in movie_names:
        if movie in movies_new['NAME'].values:
            valid_movies.append(movie)
        else:
            invalid_movies.append(movie)
    
    if invalid_movies:
        print(f"❌ 未找到以下电影: {invalid_movies}")
    
    if not valid_movies:
        print("⚠️  没有有效的输入电影")
        return None
    
    print(f"✅ 基于 {len(valid_movies)} 部电影进行推荐: {valid_movies}")
    
    # 2. 获取输入电影的索引
    input_indices = []
    for movie in valid_movies:
        movie_idx = movies_new[movies_new['NAME'] == movie].index[0]
        pos = list(movies_new.index).index(movie_idx)
        input_indices.append(pos)
    
    # 3. 计算综合相似度（根据不同策略）
    if strategy == 'average':
        # 平均相似度
        combined_similarity = np.mean([similarity[i] for i in input_indices], axis=0)
    elif strategy == 'maximum':
        # 最大相似度（与任意输入电影相似度高就推荐）
        combined_similarity = np.max([similarity[i] for i in input_indices], axis=0)
    elif strategy == 'union':
        # 并集策略：与所有输入电影都有一定相似度
        min_similarities = np.min([similarity[i] for i in input_indices], axis=0)
        combined_similarity = min_similarities
    else:
        combined_similarity = np.mean([similarity[i] for i in input_indices], axis=0)
    
    # 4. 排序和过滤
    sim_scores = list(enumerate(combined_similarity))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 排除输入电影本身，应用阈值
    sim_scores = [(i, score) for i, score in sim_scores 
                  if i not in input_indices and score >= min_similarity]
    
    if not sim_scores:
        print(f"⚠️  没有找到相似度高于 {min_similarity} 的电影")
        return None
    
    # 5. 取前topk个
    top_indices = [i for i, score in sim_scores[:topk]]
    top_scores = [score for i, score in sim_scores[:topk]]
    
    # 6. 构建结果
    recommendations = []
    for j, sim_score in zip(top_indices, top_scores):
        movie_data = movies_new.iloc[j]
        
        # 计算与每个输入电影的相似度
        individual_sims = []
        for input_idx in input_indices:
            individual_sims.append(similarity[input_idx][j])
        
        recommendations.append({
            "推荐排名": len(recommendations) + 1,
            "电影名称": movie_data['NAME'],
            "豆瓣评分": movie_data['DOUBAN_SCORE'],
            "综合相似度": f"{sim_score:.3f}",
            "导演": movie_data.get('DIRECTORS', '未知'),
            "年份": movie_data.get('YEAR', '未知'),
            "与各电影相似度": ", ".join([f"{s:.3f}" for s in individual_sims])
        })
    
    result_df = pd.DataFrame(recommendations)
    
    # 7. 打印摘要
    print(f"🎬 基于电影集合的推荐结果 (策略: {strategy})")
    print(f"📊 共找到 {len(result_df)} 部推荐电影")
    print("=" * 80)
    
    return result_df

In [ ]:
my_movies = ["肖申克的救赎", "乡愁", "霸王别姬"]
result = recommend_from_collection(my_movies, topk=8)
print(result)

my_movies 是否存在: True
my_movies 的值: ['肖申克的救赎', '乡愁', '霸王别姬']
❌ 未找到以下电影: ['肖申克的救赎']
✅ 基于 2 部电影进行推荐: ['乡愁', '霸王别姬']
🎬 基于电影集合的推荐结果 (策略: average)
📊 共找到 8 部推荐电影
   推荐排名     电影名称  豆瓣评分  综合相似度         导演      年份       与各电影相似度
0     1     一代宗师   7.9  0.767        王家卫  2013.0  0.735, 0.799
1     2    绅士爱美人   7.8  0.766    霍华德·霍克斯  1953.0  0.769, 0.763
2     3   他和她的故事   7.8  0.763      R·巴尔基  2016.0  0.776, 0.750
3     4  尤里西斯的凝视   9.0  0.762  西奥·安哲罗普洛斯  1995.0  0.815, 0.709
4     5     鸳梦重温   8.6  0.760     茂文·勒鲁瓦  1942.0  0.774, 0.747
5     6  金玉良缘红楼梦   7.4  0.760        李翰祥  1977.0  0.726, 0.794
6     7    刺客聂隐娘   7.3  0.757        侯孝贤  2015.0  0.707, 0.808
7     8     将来的事   8.2  0.757   米娅·汉森-洛夫  2016.0  0.753, 0.760


## 根据电影集进行推荐，输出为文本，美化排版

In [54]:
from tabulate import tabulate

def print_pretty_table(df, title="", table_format="fancy_grid"):
    """使用tabulate打印美观的表格"""
    if df is None or df.empty:
        print("暂无数据")
        return
    
    # 设置表格格式
    table_formats = {
        'simple': 'simple',
        'grid': 'grid', 
        'fancy_grid': 'fancy_grid',
        'github': 'github',
        'pretty': 'pretty'
    }
    
    format_choice = table_formats.get(table_format, 'fancy_grid')
    
    # 打印标题
    if title:
        print(f"\n{'='*80}")
        print(f"🎬 {title}")
        print(f"{'='*80}")
    
    # 转换并打印表格
    table_str = tabulate(df, headers='keys', tablefmt=format_choice, 
                        showindex=False, numalign="center", stralign="left")
    print(table_str)
    print(f"{'='*80}")

# 美化版推荐函数
def recommend_pretty(movie_names, topk=10, min_similarity=0.0, strategy='average', 
                    table_format="fancy_grid"):
    """完整的美化版推荐函数"""
    
    # 输入验证
    valid_movies = [m for m in movie_names if m in movies_new['NAME'].values]
    invalid_movies = [m for m in movie_names if m not in movies_new['NAME'].values]
    
    if invalid_movies:
        print(f"❌ 未找到以下电影: {', '.join(invalid_movies)}")
    
    if not valid_movies:
        print("⚠️  没有有效的输入电影")
        return None
    
    print(f"✅ 基于 {len(valid_movies)} 部电影进行推荐: {', '.join(valid_movies)}")
    
    # 计算推荐（使用之前的逻辑）
    input_indices = []
    for movie in valid_movies:
        movie_idx = movies_new[movies_new['NAME'] == movie].index[0]
        pos = list(movies_new.index).index(movie_idx)
        input_indices.append(pos)
    
    if strategy == 'average':
        combined_similarity = np.mean([similarity[i] for i in input_indices], axis=0)
    elif strategy == 'maximum':
        combined_similarity = np.max([similarity[i] for i in input_indices], axis=0)
    else:
        combined_similarity = np.mean([similarity[i] for i in input_indices], axis=0)
    
    # 排序和过滤
    sim_scores = list(enumerate(combined_similarity))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = [(i, score) for i, score in sim_scores 
                  if i not in input_indices and score >= min_similarity]
    
    if not sim_scores:
        print(f"⚠️  没有找到相似度高于 {min_similarity} 的电影")
        return None
    
    # 构建结果
    top_indices = [i for i, score in sim_scores[:topk]]
    top_scores = [score for i, score in sim_scores[:topk]]
    
    recommendations = []
    for j, sim_score in zip(top_indices, top_scores):
        movie_data = movies_new.iloc[j]
        
        recommendations.append({
            "排名": len(recommendations) + 1,
            "电影名称": movie_data['NAME'],
            "豆瓣评分": float(movie_data['DOUBAN_SCORE']),
            "相似度": float(f"{sim_score:.3f}"),
            "导演": str(movie_data.get('DIRECTORS', '未知'))[:15],
            "年份": str(movie_data.get('YEAR', '未知')),
            "主演": str(movie_data.get('ACTORS', '未知'))[:20] + "..."
        })
    
    result_df = pd.DataFrame(recommendations)
    
    # 美化输出
    title = f"电影推荐结果 | 策略: {strategy} | 数量: {topk}"
    print_pretty_table(result_df, title, table_format)
    
    return result_df

In [ ]:
my_movies = ["肖申克的救赎", "乡愁", "霸王别姬"]
result1 = recommend_pretty(my_movies, topk=5, table_format="fancy_grid")

❌ 未找到以下电影: 肖申克的救赎
✅ 基于 2 部电影进行推荐: 乡愁, 霸王别姬

🎬 电影推荐结果 | 策略: average | 数量: 5
╒════════╤════════════════╤════════════╤══════════╤═══════════════════╤════════╤════════════════════════════════════════╕
│  排名  │ 电影名称       │  豆瓣评分  │  相似度  │ 导演              │  年份  │ 主演                                   │
╞════════╪════════════════╪════════════╪══════════╪═══════════════════╪════════╪════════════════════════════════════════╡
│   1    │ 一代宗师       │    7.9     │  0.767   │ 王家卫            │  2013  │ 梁朝伟/章子怡/张震/宋慧乔/赵本山/小... │
├────────┼────────────────┼────────────┼──────────┼───────────────────┼────────┼────────────────────────────────────────┤
│   2    │ 绅士爱美人     │    7.8     │  0.766   │ 霍华德·霍克斯     │  1953  │ 简·拉塞尔/玛丽莲·梦露/查尔斯·科本/...  │
├────────┼────────────────┼────────────┼──────────┼───────────────────┼────────┼────────────────────────────────────────┤
│   3    │ 他和她的故事   │    7.8     │  0.763   │ R·巴尔基          │  2016  │ 阿琼·卡普尔/卡琳娜·卡普尔/贾雅·巴杜... │
├────────┼────────────────┼────────────

## 单电影输入，GUI版

In [58]:
import tkinter as tk
from tkinter import ttk, scrolledtext, messagebox
import numpy as np
import pandas as pd

class MovieRecommendationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🎬 电影推荐系统")
        self.root.geometry("800x600")
        
        self.setup_ui()
    
    def setup_ui(self):
        # 标题
        title_label = tk.Label(
            self.root, 
            text="🎬 豆mini智能电影推荐系统", 
            font=("Arial", 16, "bold")
        )
        title_label.pack(pady=20)
        
        # 输入区域
        input_frame = tk.Frame(self.root)
        input_frame.pack(pady=10)
        
        tk.Label(input_frame, text="输入电影名称:", font=("Arial", 12)).pack(side=tk.LEFT, padx=5)
        
        self.movie_entry = tk.Entry(input_frame, width=30, font=("Arial", 12))
        self.movie_entry.pack(side=tk.LEFT, padx=5)
        self.movie_entry.bind('<Return>', lambda event: self.recommend_movie())
        
        # 推荐数量选择
        tk.Label(input_frame, text="推荐数量:", font=("Arial", 12)).pack(side=tk.LEFT, padx=(20,5))
        
        self.topk_var = tk.StringVar(value="5")
        topk_spinbox = tk.Spinbox(
            input_frame, 
            from_=1, 
            to=20, 
            width=5, 
            textvariable=self.topk_var,
            font=("Arial", 12)
        )
        topk_spinbox.pack(side=tk.LEFT, padx=5)
        
        # 推荐按钮
        recommend_btn = tk.Button(
            input_frame,
            text="开始推荐",
            command=self.recommend_movie,
            font=("Arial", 12, "bold"),
            bg='#3498db',
            fg='white',
            padx=20
        )
        recommend_btn.pack(side=tk.LEFT, padx=10)
        
        # 结果显示区域
        result_frame = tk.Frame(self.root)
        result_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        # 创建文本框用于显示结果
        self.result_text = scrolledtext.ScrolledText(
            result_frame,
            wrap=tk.WORD,
            font=("Arial", 11),
            padx=10,
            pady=10
        )
        self.result_text.pack(fill=tk.BOTH, expand=True)
        
        # 状态栏
        self.status_var = tk.StringVar(value="就绪 - 请输入电影名称")
        status_bar = tk.Label(
            self.root, 
            textvariable=self.status_var,
            relief=tk.SUNKEN, 
            anchor=tk.W,
            font=("Arial", 10),
            bg='#f0f0f0'
        )
        status_bar.pack(side=tk.BOTTOM, fill=tk.X)
        
        # 添加一些提示
        self.show_welcome_message()
    
    def show_welcome_message(self):
        welcome_text = """✨ 欢迎使用电影推荐系统 ✨

使用方法：
1. 在输入框中输入电影名称
2. 选择推荐数量（1-20部）
3. 点击「开始推荐」或按回车键

示例电影：
• 肖申克的救赎
• 霸王别姬  
• 阿甘正传
• 泰坦尼克号
• 这个杀手不太冷

提示：确保输入正确的电影名称
"""
        self.result_text.insert(tk.END, welcome_text)
        self.result_text.config(state=tk.DISABLED)
    
    def recommend_movie(self):
        movie_name = self.movie_entry.get().strip()
        if not movie_name:
            messagebox.showwarning("输入错误", "请输入电影名称")
            return
        
        try:
            topk = int(self.topk_var.get())
        except ValueError:
            messagebox.showwarning("输入错误", "推荐数量必须是数字")
            return
        
        self.status_var.set("正在搜索...")
        self.root.update()
        
        # 清空之前的结果
        self.result_text.config(state=tk.NORMAL)
        self.result_text.delete(1.0, tk.END)
        
        # 调用推荐函数
        try:
            result_df = self.get_recommendations(movie_name, topk)
            
            if result_df is None:
                self.result_text.insert(tk.END, f"❌ 未找到电影《{movie_name}》\n\n")
                self.result_text.insert(tk.END, "💡 提示：请检查电影名称是否正确")
                self.status_var.set(f"未找到电影: {movie_name}")
            else:
                # 显示推荐结果
                self.display_recommendations(movie_name, result_df, topk)
                self.status_var.set(f"推荐完成 - 找到 {len(result_df)} 部相似电影")
                
        except Exception as e:
            self.result_text.insert(tk.END, f"❌ 发生错误: {str(e)}")
            self.status_var.set("发生错误")
        
        self.result_text.config(state=tk.DISABLED)
    
    def get_recommendations(self, movie_name, topk=5):
        """推荐函数"""
        try:
            # 查找电影
            movie_matches = movies_new[movies_new['NAME'] == movie_name]
            if len(movie_matches) == 0:
                return None
            
            # 获取位置
            movie_idx = movie_matches.index[0]
            pos = list(movies_new.index).index(movie_idx)
            
            # 计算相似度
            sims = similarity[pos]
            cand = np.argsort(-sims)  # 降序排列
            cand = cand[cand != pos]  # 排除自身
            cand = cand[:topk]  # 取前topk个
            
            # 构建结果
            recs = []
            for j in cand:
                movie_data = movies_new.iloc[j]
                rec = {
                    "电影名": movie_data['NAME'],
                    "豆瓣评分": movie_data['DOUBAN_SCORE'],
                    "相似度": sims[j]
                }
                
                # 添加可选信息
                if 'DIRECTORS' in movies_new.columns:
                    rec["导演"] = movie_data.get('DIRECTORS', '未知')
                if 'YEAR' in movies_new.columns:
                    rec["年份"] = movie_data.get('YEAR', '未知')
                
                recs.append(rec)
            
            return pd.DataFrame(recs)
            
        except Exception as e:
            print(f"推荐过程中出错: {e}")
            return None
    
    def display_recommendations(self, original_movie, result_df, topk):
        """在文本框中显示推荐结果"""
        # 获取原始电影信息
        try:
            original_movie_data = movies_new[movies_new['NAME'] == original_movie].iloc[0]
            original_score = original_movie_data['DOUBAN_SCORE']
        except:
            original_score = "未知"
        
        # 构建输出文本
        output = f"""
🎬 电影推荐结果
{'='*50}
原始电影: 《{original_movie}》 (豆瓣评分: {original_score})
推荐数量: {topk} 部
{'='*50}

"""
        
        for i, row in result_df.iterrows():
            similarity_score = row['相似度']
            
            # 根据相似度选择符号
            if similarity_score > 0.8:
                sim_symbol = "【高相似】"
            elif similarity_score > 0.6:
                sim_symbol = "【中相似】" 
            elif similarity_score > 0.4:
                sim_symbol = "【低相似】"
            else:
                sim_symbol = "【一般】"
            
            output += f"{i+1:2d}. {sim_symbol} 《{row['电影名']}》\n"
            output += f"    评分: {row['豆瓣评分']} | 相似度: {similarity_score:.3f}\n"
            
            if '导演' in row and row['导演'] != '未知':
                output += f"    导演: {row['导演']}\n"
            if '年份' in row and row['年份'] != '未知':
                output += f"    年份: {row['年份']}\n"
            
            if i < len(result_df) - 1:
                output += "    " + "-" * 40 + "\n"
        
        output += f"""
{'='*50}
推荐完成！共找到 {len(result_df)} 部相似电影
"""
        
        self.result_text.insert(tk.END, output)

# 启动应用程序
def start_gui():
    root = tk.Tk()
    app = MovieRecommendationApp(root)
    root.mainloop()

# 运行GUI
if __name__ == "__main__":
    # 确保这些变量已经定义
    if 'movies_new' not in globals() or 'similarity' not in globals():
        print("错误: 请先运行数据准备代码")
    else:
        start_gui()

## 输入为电影集，GUI版

In [76]:
import tkinter as tk
from tkinter import ttk, scrolledtext, messagebox
import numpy as np
import pandas as pd

class MovieCollectionRecommendationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🎬 电影集合推荐系统")
        self.root.geometry("900x700")
        
        self.setup_ui()
    
    def setup_ui(self):
        # 标题
        title_label = tk.Label(
            self.root, 
            text="🎬 豆MINI智能电影集合推荐系统", 
            font=("Arial", 16, "bold")
        )
        title_label.pack(pady=20)
        
        # 输入区域
        input_frame = tk.Frame(self.root)
        input_frame.pack(pady=10, fill=tk.X, padx=20)
        
        # 电影集合输入
        tk.Label(input_frame, text="输入电影集合:", font=("Arial", 12)).grid(row=0, column=0, sticky="w", pady=5)
        
        input_help = tk.Label(input_frame, text="(多个电影用逗号、分号或换行分隔)", font=("Arial", 9), fg="gray")
        input_help.grid(row=0, column=1, sticky="w", pady=5)
        
        self.movies_text = scrolledtext.ScrolledText(input_frame, height=4, width=60, font=("Arial", 11))
        self.movies_text.grid(row=1, column=0, columnspan=3, sticky="ew", pady=5)
        
        # 推荐设置区域
        settings_frame = tk.Frame(self.root)
        settings_frame.pack(pady=10, fill=tk.X, padx=20)
        
        tk.Label(settings_frame, text="推荐数量:", font=("Arial", 12)).pack(side=tk.LEFT, padx=5)
        
        self.topk_var = tk.StringVar(value="10")
        topk_spinbox = tk.Spinbox(
            settings_frame, 
            from_=1, 
            to=20, 
            width=5, 
            textvariable=self.topk_var,
            font=("Arial", 12)
        )
        topk_spinbox.pack(side=tk.LEFT, padx=5)
        
        # 推荐策略选择
        tk.Label(settings_frame, text="推荐策略:", font=("Arial", 12)).pack(side=tk.LEFT, padx=(20,5))
        
        self.strategy_var = tk.StringVar(value="average")
        strategy_combo = ttk.Combobox(
            settings_frame,
            textvariable=self.strategy_var,
            values=["average", "maximum", "union"],
            state="readonly",
            width=10
        )
        strategy_combo.pack(side=tk.LEFT, padx=5)
        
        # 策略说明
        strategy_help = tk.Label(settings_frame, text="(平均相似度 | 最大相似度 | 并集相似度)", font=("Arial", 9), fg="gray")
        strategy_help.pack(side=tk.LEFT, padx=5)
        
        # 按钮区域
        button_frame = tk.Frame(self.root)
        button_frame.pack(pady=10)
        
        recommend_btn = tk.Button(
            button_frame,
            text="开始推荐",
            command=self.recommend_movies,
            font=("Arial", 12, "bold"),
            bg='#3498db',
            fg='white',
            padx=30,
            pady=5
        )
        recommend_btn.pack(side=tk.LEFT, padx=10)
        
        clear_btn = tk.Button(
            button_frame,
            text="清空输入",
            command=self.clear_input,
            font=("Arial", 12),
            bg='#95a5a6',
            fg='white',
            padx=20,
            pady=5
        )
        clear_btn.pack(side=tk.LEFT, padx=10)
        
        # 结果显示区域
        result_frame = tk.Frame(self.root)
        result_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        # 创建文本框用于显示结果
        self.result_text = scrolledtext.ScrolledText(
            result_frame,
            wrap=tk.WORD,
            font=("Arial", 11),
            padx=10,
            pady=10
        )
        self.result_text.pack(fill=tk.BOTH, expand=True)
        
        # 状态栏
        self.status_var = tk.StringVar(value="就绪 - 请输入电影集合")
        status_bar = tk.Label(
            self.root, 
            textvariable=self.status_var,
            relief=tk.SUNKEN, 
            anchor=tk.W,
            font=("Arial", 10),
            bg='#f0f0f0'
        )
        status_bar.pack(side=tk.BOTTOM, fill=tk.X)
        
        # 添加一些提示
        self.show_welcome_message()
    
    def show_welcome_message(self):
        welcome_text = """✨ 欢迎使用电影集合推荐系统 ✨

使用方法：
1. 在输入框中输入多个电影名称（用逗号、分号或换行分隔）
2. 选择推荐数量和策略
3. 点击「开始推荐」按钮

推荐策略说明：
• 平均相似度：计算与所有输入电影的平均相似度
• 最大相似度：取与任意输入电影的最大相似度  
• 并集相似度：要求与所有输入电影都有一定相似度

示例输入：
肖申克的救赎, 霸王别姬, 阿甘正传
或：
肖申克的救赎
霸王别姬
阿甘正传

提示：确保输入正确的电影名称
"""
        self.result_text.insert(tk.END, welcome_text)
        self.result_text.config(state=tk.DISABLED)
    
    def clear_input(self):
        """清空输入框"""
        self.movies_text.delete(1.0, tk.END)
        self.status_var.set("输入已清空")
    
    def parse_movie_input(self, input_text):
        """解析用户输入的电影集合"""
        # 替换各种分隔符为逗号
        input_text = input_text.replace(';', ',').replace('，', ',').replace('\n', ',')
        # 分割并清理
        movies = [movie.strip() for movie in input_text.split(',') if movie.strip()]
        return movies
    
    def recommend_movies(self):
        """基于电影集合进行推荐"""
        input_text = self.movies_text.get(1.0, tk.END).strip()
        if not input_text:
            messagebox.showwarning("输入错误", "请输入电影名称")
            return
        
        try:
            topk = int(self.topk_var.get())
            strategy = self.strategy_var.get()
        except ValueError:
            messagebox.showwarning("输入错误", "推荐数量必须是数字")
            return
        
        # 解析输入的电影集合
        input_movies = self.parse_movie_input(input_text)
        if not input_movies:
            messagebox.showwarning("输入错误", "未找到有效的电影名称")
            return
        
        self.status_var.set(f"正在处理 {len(input_movies)} 部电影...")
        self.root.update()
        
        # 清空之前的结果
        self.result_text.config(state=tk.NORMAL)
        self.result_text.delete(1.0, tk.END)
        
        # 调用推荐函数
        try:
            result_df = self.get_collection_recommendations(input_movies, topk, strategy)
            
            if result_df is None:
                self.result_text.insert(tk.END, f"❌ 推荐失败\n\n")
                self.result_text.insert(tk.END, "💡 提示：请检查电影名称是否正确")
                self.status_var.set("推荐失败")
            else:
                # 显示推荐结果
                self.display_collection_recommendations(input_movies, result_df, topk, strategy)
                self.status_var.set(f"推荐完成 - 找到 {len(result_df)} 部推荐电影")
                
        except Exception as e:
            self.result_text.insert(tk.END, f"❌ 发生错误: {str(e)}")
            self.status_var.set("发生错误")
        
        self.result_text.config(state=tk.DISABLED)
    
    def get_collection_recommendations(self, movie_names, topk=10, strategy='average'):
        """基于电影集合的推荐函数"""
        try:
            # 验证输入电影
            valid_movies = []
            invalid_movies = []
            input_indices = []
            
            for movie in movie_names:
                movie_matches = movies_new[movies_new['NAME'] == movie]
                if len(movie_matches) > 0:
                    valid_movies.append(movie)
                    movie_idx = movie_matches.index[0]
                    pos = list(movies_new.index).index(movie_idx)
                    input_indices.append(pos)
                else:
                    invalid_movies.append(movie)
            
            if not valid_movies:
                return None
            
            # 记录无效电影（用于显示）
            self.invalid_movies = invalid_movies
            
            # 根据策略计算综合相似度
            if strategy == 'average':
                # 平均相似度
                combined_similarity = np.mean([similarity[i] for i in input_indices], axis=0)
            elif strategy == 'maximum':
                # 最大相似度
                combined_similarity = np.max([similarity[i] for i in input_indices], axis=0)
            elif strategy == 'union':
                # 并集相似度（最小相似度）
                combined_similarity = np.min([similarity[i] for i in input_indices], axis=0)
            else:
                combined_similarity = np.mean([similarity[i] for i in input_indices], axis=0)
            
            # 排序和过滤
            sim_scores = list(enumerate(combined_similarity))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            
            # 排除输入电影本身
            sim_scores = [(i, score) for i, score in sim_scores if i not in input_indices]
            
            if not sim_scores:
                return None
            
            # 取前topk个
            top_indices = [i for i, score in sim_scores[:topk]]
            top_scores = [score for i, score in sim_scores[:topk]]
            
            # 构建结果
            recs = []
            for j, sim_score in zip(top_indices, top_scores):
                movie_data = movies_new.iloc[j]
                rec = {
                    "电影名": movie_data['NAME'],
                    "豆瓣评分": movie_data['DOUBAN_SCORE'],
                    "综合相似度": sim_score
                }
                
                # 计算与每个输入电影的单独相似度
               # individual_sims = [similarity[input_idx][j] for input_idx in input_indices]
               # rec["各电影相似度"] = individual_sims
                
                # 添加可选信息
                if 'DIRECTORS' in movies_new.columns:
                    rec["导演"] = movie_data.get('DIRECTORS', '未知')
                if 'YEAR' in movies_new.columns:
                    rec["年份"] = movie_data.get('YEAR', '未知')
                
                recs.append(rec)
            
            return pd.DataFrame(recs)
            
        except Exception as e:
            print(f"推荐过程中出错: {e}")
            return None
    
    def display_collection_recommendations(self, input_movies, result_df, topk, strategy):
        """显示电影集合的推荐结果"""
        # 策略名称映射
        strategy_names = {
            'average': '平均相似度',
            'maximum': '最大相似度', 
            'union': '并集相似度'
        }
        
        strategy_name = strategy_names.get(strategy, strategy)
        
        # 构建输出文本
        output = f"""
🎬 电影集合推荐结果
{'='*60}
📚 输入电影: {', '.join(input_movies)}
🎯 推荐策略: {strategy_name}
📊 推荐数量: {topk} 部
{'='*60}

"""
        
        # 显示无效电影警告
        if hasattr(self, 'invalid_movies') and self.invalid_movies:
            output += f"⚠️  未找到以下电影: {', '.join(self.invalid_movies)}\n\n"
        
        # 显示推荐结果
        for i, row in result_df.iterrows():
            similarity_score = row['综合相似度']
            
            # 根据相似度选择符号
            if similarity_score > 0.7:
                sim_symbol = "【高相似】"
            elif similarity_score > 0.5:
                sim_symbol = "【中相似】" 
            elif similarity_score > 0.3:
                sim_symbol = "【低相似】"
            else:
                sim_symbol = "【一般】"
            
            output += f"{i+1:2d}. {sim_symbol} 《{row['电影名']}》\n"
            output += f"    ⭐ 评分: {row['豆瓣评分']} | 📊 综合相似度: {similarity_score:.3f}\n"
            
            # 显示与每个输入电影的相似度
            if '各电影相似度' in row:
                individual_sims = row['各电影相似度']
                sim_details = " | ".join([f"{sim:.3f}" for sim in individual_sims])
                output += f"    🔍 各电影相似度: {sim_details}\n"
            
            if '导演' in row and row['导演'] != '未知':
                output += f"    🎥 导演: {row['导演']}\n"
            if '年份' in row and row['年份'] != '未知':
                output += f"    📅 年份: {row['年份']}\n"
            
            if i < len(result_df) - 1:
                output += "    " + "─" * 50 + "\n"
        
        output += f"""
{'='*60}
✅ 推荐完成！基于 {len(input_movies)} 部输入电影，找到 {len(result_df)} 部推荐电影
📈 平均相似度: {result_df['综合相似度'].mean():.3f}
"""
        
        self.result_text.insert(tk.END, output)

# 启动应用程序
def start_collection_gui():
    root = tk.Tk()
    app = MovieCollectionRecommendationApp(root)
    root.mainloop()

# 运行GUI
if __name__ == "__main__":
    # 确保这些变量已经定义
    if 'movies_new' not in globals() or 'similarity' not in globals():
        print("错误: 请先运行数据准备代码")
    else:
        start_collection_gui()

### 后续需添加导演和时代标签等关系网